# Advent of code 2020

This notebook contains my (somewhat documented) solutions to advent of code 2020. For
each day, I've tried to summarize the solutions into short explanation, understandable
code blocks and, if needed, have added some extra information in code docstrings.

If you want to tinker around with the solutions, you can install needed libraries
with either [poetry](https://python-poetry.org/) or just plain `pip install .`. Some
extra setup is also needed for the awesome [advent-of-code-data](
https://pypi.org/project/advent-of-code-data/) library, which I'm using for getting
input. See project docs for information.

## Setup and shared functions

This first block is used to import all libraries, define all special function and
create all types needed in the notebook. These are mainly for parsing data into
different forms.

In [1]:
import re
from aocd import get_data
from collections import defaultdict, deque
from itertools import combinations, count
from math import prod
from typing import Any, Iterable, Callable, Optional, Literal, Iterator
from dataclasses import dataclass
from sympy.ntheory.modular import crt
from sympy.ntheory import discrete_log
from networkx import DiGraph
from networkx.algorithms import dag

Vector = tuple[int, int]
Mode = Literal["first", "second"]
Solution = tuple[Any, Any]

def data(day: int) -> str:
    """Get days input data as one string."""
    return get_data(day=day, year=2020)

def data_to_nums(day: int) -> list[int]:
    """Get days input data as numbers."""
    return [int(x) for x in re.findall(r"(\d+)", get_data(day=day, year=2020))]

def data_to_lines(day: int) -> list[str]:
    """Get days input data lines as list of strings."""
    return get_data(day=day, year=2020).splitlines()

def data_to_blocks(day: int) -> list[str]:
    """Get days input data blocks that are separated by empty new lines."""
    return [block for block in get_data(day=day, year=2020).split("\n\n")]

def get_next(iterable: Iterable, condition: Callable) -> Any:
    """Get first object from iterable that matches given condition."""
    return next(thing for thing in iterable if condition(thing))

def run_day(day_number: int) -> Solution:
    """Return solutions for given day."""
    day_func = globals()[f"day_{day_number}"]
    return day_func()
    
def print_day(day_number: int) -> None:
    """Print solutions for given day."""
    first, second = run_day(day_number)
    print("Part 1:", first)
    print("Part 2:", second)


class CellularAutomaton:
    """Class to simulate different cellular automaton -style problems."""
    pass

## Day 1

**Task**: find the entries that sum to 2020.

Input size is quite small, just hundred numbers, so brute-forcing through every possible
combination is fast enough. Solve-function gets first $k$-tuple where sum of tuple
matches given goal.

In [2]:
def day_1() -> Solution:
    numbers = data_to_nums(day=1)

    def solve(goal: int, number_of_entries: int) -> int:
        return prod(get_next(combinations(numbers, number_of_entries),
                                  lambda *combination: sum(*combination) == goal))
    
    first = solve(goal=2020, number_of_entries=2)
    second = solve(goal=2020, number_of_entries=3)
    
    return first, second

In [3]:
print_day(1)

Part 1: 1010299
Part 2: 42140160


## Day 2

**Task**: find amount of passwords that match their counterpart rule.

Ah, classic aocd-style input format! Luckily, 
[regex](https://en.wikipedia.org/wiki/Regular_expression) is here to help.

For the first part, occurrences of char in string is needed. The second part is handled
with [exclusive or (XOR)](https://en.wikipedia.org/wiki/Exclusive_or) since char at
either, but not both positions must equal to
char in rule.

In [4]:
def day_2() -> Solution:
    lines = data_to_lines(day=2)
    regex = re.compile(r"(\d+)-(\d+) ([a-z]): ([a-z]+)")
    first, second = 0, 0

    for line in lines:
        low, high, char, string = regex.match(line).groups()
        low, high = int(low), int(high)
        if low <= string.count(char) <= high:
            first += 1
        if (string[low - 1] == char) ^ (string[high - 1] == char):
            second += 1
            
    return first, second

In [5]:
print_day(2)

Part 1: 524
Part 2: 485


## Day 3

**Task**: count the number of trees encountered while traversing the map.

Instead of really traversing through given map, the `count_trees` function uses `range`
and `count` generators zipped into tuples to create all traversed points. Then, it's
easy to count encountered trees.

In [6]:
def day_3() -> Solution:
    grid = data_to_lines(day=3)

    def count_trees(slope: Vector):
        right, down = slope
        height, width = len(grid), len(grid[0])
        return sum(
            grid[y][x % width] == "#"
            for y, x in zip(range(0, height, down), count(start=0, step=right))
        )

    slopes = [(3,1), (1, 1), (5, 1), (7, 1), (1, 2)]
    counts = [count_trees(slope=slope) for slope in slopes]
    
    return counts[0], prod(counts)

In [7]:
print_day(3)

Part 1: 280
Part 2: 4355551200


## Day 4

**Task**: count the number of valid passports

In the first part, it's enough to check that all ids in validators are present in a
passport. For the second part, all validator lambdas are also executed against passport
values.

In [8]:
Validator = Callable[[str], bool]

def day_4() -> Solution:
    passports = data_to_blocks(day=4)
    validators: dict[str, Validator] = {
        "byr": lambda v: 1920 <= int(v) <= 2002,
        "iyr": lambda v: 2010 <= int(v) <= 2020,
        "eyr": lambda v: 2020 <= int(v) <= 2030,
        "hgt": lambda v: (
            "cm" in v
            and 150 <= int(v[:-2]) <= 193
            or "in" in v
            and 59 <= int(v[:-2]) <= 76
        ),
        "hcl": lambda v: bool(re.fullmatch(r"^#[0-9a-f]{6}$", v)),
        "ecl": lambda v: v in ["amb", "blu", "brn", "gry", "grn", "hzl", "oth"],
        "pid": lambda v: bool(re.fullmatch(r"^[0-9]{9}$", v)),
    }
    regex = re.compile("(\w+):(\S+)")
    first, second = 0, 0
    
    for passport in passports:
            fields = dict(regex.findall(passport))
            if set(validators).issubset(set(fields)):
                first += 1
                second += all(validate(fields[field_id])
                              for field_id, validate 
                              in validators.items())
                
    return first, second

In [9]:
print_day(4)

Part 1: 192
Part 2: 101


## Day 5

**Task**: Find out the highest seat ID and your own seat ID.

There's a nice catch in the puzzle: char pairs F/L and B/R aren't really different,
since each can be substituted with 0 or 1. The result is a [binary number
](https://en.wikipedia.org/wiki/Binary_number) string which can be casted to int.

In the second part, helper function finds the first seat id that matches given
conditions.

In [10]:
def day_5() -> Solution:
    lines = data_to_lines(day=5)

    table = str.maketrans("FBLR", "0101")
    seat_ids = {int(line.translate(table), base=2) for line in lines}
    
    first = max(seat_ids)
    second = get_next(range(1, first), 
                      lambda seat: seat not in seat_ids 
                      and seat - 1 in seat_ids 
                      and seat + 1 in seat_ids)
    
    return first, second

In [11]:
print_day(5)

Part 1: 858
Part 2: 557


## Day 6

**Task**: Count the number of questions to which anyone/everyone answered "yes".

In the first part, the question is basically "how many unique alphabets there are
between newlines?". In the second part, question is "which alphabets are present on all
lines between newlines?", which can be handled with [set intersections
](https://en.wikipedia.org/wiki/Intersection_(set_theory)).

In [12]:
def day_6() -> Solution:
    groups = data_to_blocks(day=6)
    first, second = 0, 0

    for answers in groups:
        answer_ids = {answer for answer in answers if answer.isalpha()}
        first += len(answer_ids)
        for person in answers.split("\n"):
            answer_ids &= set(person)
        second += len(answer_ids)
        
    return first, second

In [13]:
print_day(6)

Part 1: 6532
Part 2: 3427


## Day 7

**Task**: Which bags can contain "shiny gold" bag / how many bags does "shiny gold" bag
contain?

A bit trickier one! One helpful observation is that the rules define a graph, a
[directed acyclic graph](https://en.wikipedia.org/wiki/Directed_acyclic_graph) (DAG) 
to be precise. So, the excellent network library [NetworkX](https://networkx.org/) can
be used.

Initially, graph contains weighted edges from container bag to content bags. The first 
part is solved by turning edges around to answer the question "which bags can contain
this bag". This way it's easy to walk graph through from the "shiny gold" node.

In second part, [depth-first-search](https://en.wikipedia.org/wiki/Depth-first_search)
is used to sum bag amounts from the innermost bags until "shiny gold" is reached.

In [14]:
Node = str
Edge = tuple[Node, int]
Graph = dict[Node, list[Edge]]

def day_7():
    rules = data_to_lines(day=7)
    graph = DiGraph()

    def create_graph() -> None:
        regex = re.compile(r"(\d+) ([\w\s]+(?= ))")
        for rule in rules:
            container, content = rule.split(" bags contain ")
            graph.add_weighted_edges_from(
                ((container, bag, int(amount)) 
                for amount, bag 
                in regex.findall(content))
            )

    def dfs(bag: str) -> int:
        return sum(
            dfs(neighbour) * graph[bag][neighbour]["weight"] 
            for neighbour 
            in graph[bag]
        ) + 1

    create_graph()
    first = len(dag.descendants(graph.reverse(), "shiny gold"))
    second = dfs("shiny gold") - 1
    
    return first, second

In [15]:
print_day(7)

Part 1: 372
Part 2: 8015


## Day 8

**Task**: Where the given semi-assembly program loops? / Change one instruction
to make program terminate successfully.

This reminded me of [Intcode computer](https://adventofcode.com/2019/day/2) tasks from
AoC 2019, so I decided to just simulate the given program.

In the first part, loop breaks once some instruction is visited for the second time. In
the second part, generator function is used to create potential tapes where either "jmp"
or "nop" is replaced. Function get_first returns value from the first tape that goes
beyond its index.

In [16]:
Instruction = tuple[str, int]
Tape = list[Instruction]

def day_8():
    instructions = re.findall(r"(\w+) ([+-]\d+)", data(day=8))
    tape = [(op, int(arg)) for op, arg in instructions]

    @dataclass
    class Comp:
        tape: Tape
        accumulator: int = 0
        head: int = 0

        def run(self, mode: Mode = "first") -> Optional[int]:
            seen: set[int] = set()
            while self.head not in seen:
                seen.add(self.head)
                try:
                    op, arg = self.tape[self.head]
                except IndexError:
                    return self.accumulator
                if op == "acc":
                    self.accumulator += arg
                    self.head += 1
                elif op == "jmp":
                    self.head += arg
                elif op == "nop":
                    self.head += 1
            if mode == "first":
                return self.accumulator

    def potential_tapes(broken_tape: Tape) -> Iterator[Tape]:
        replace = {"jmp": "nop", "nop": "jmp"}
        for i, (op, arg) in enumerate(broken_tape):
            if op in replace:
                yield [*broken_tape[:i], (replace[op], arg), *broken_tape[i + 1:]]
                
    first = Comp(tape).run()
    second = get_next((Comp(potential_tape).run(mode="second")
                       for potential_tape in potential_tapes(tape)),
                      lambda result: result is not None)
    
    return first, second

In [17]:
print_day(8)

Part 1: 2034
Part 2: 672


## Day 9

**Task**: What is the first number that isn't sum of any two numbers of the 25
numbers before it? / Find a contiguous subset of numbers that sum up to that first
number.

Time to brute-force again! In the first part, sum_in_interval checks if number at
an index $i$ is a sum of any combination of previous 25 numbers. Number at first index
satisfying this property is the answer.

In the second part I decided to put a little of effort to produce an $O(n)$ solution.
Solution uses a [sliding window](
https://www.geeksforgeeks.org/window-sliding-technique/) and deque in order to have each
number added/removed from `sum_nums` max one time.

In [18]:
def day_9():
    numbers = data_to_nums(day=9)
    n, offset = len(numbers), 25

    def sum_in_interval(start_index: int, goal_index: int) -> bool:
        return any(sum(combination) == numbers[goal_index] for combination in
                   combinations(numbers[start_index:goal_index], 2))

    def find_sum(goal: int) -> Optional[int]:
        sum_nums, sum_total = deque(), 0
        for num in numbers:
            if sum_total < goal:
                sum_nums.append(num)
                sum_total += num
            if sum_total == goal and len(sum_nums) >= 2:
                return min(sum_nums) + max(sum_nums)
            while sum_total > goal:
                sum_total -= sum_nums.popleft()

    invalid_number_index = get_next(range(offset, n), lambda index: 
                                    not sum_in_interval(start_index=index-offset,
                                                        goal_index=index)
                                   )
    first = numbers[invalid_number_index]
    second = find_sum(goal=first)
    
    return first, second

In [19]:
print_day(9)

Part 1: 542529149
Part 2: 75678618


## Day 10

**Task**: Count differences between numbers / count total numbers of ways to arrange
numbers up to `n`th number.

It helps here to sort the given joltage array first. After that, both parts can be
solved in $O(n)$ by iterating through the sorted array. 

For the first part, differences between consecutive numbers are calculated. The 
second part can be solved with 
[dynamic programming](https://en.wikipedia.org/wiki/Dynamic_programming): since
one can add either adapter with $k+1$, $k+2$ or $k+3$ joltages after an adapter
with $k$ joltages, amount of adapters at some $k$ is based on amounts before it.
Answer is the amount of possible ways to sum up to charger with largest joltage.

In [20]:
def day_10():
    joltages = sorted(data_to_nums(day=10))

    ways, diffs = defaultdict(int), defaultdict(int)
    prev = 0
    ways[0] = 1
    for joltage in joltages:
        ways[joltage] = sum(ways[joltage - i] for i in [1, 2, 3])
        diffs[joltage - prev] += 1
        prev = joltage
    diffs[3] += 1
    
    first = diffs[1] * diffs[3]
    second = ways[joltages[-1]]
    
    return first, second

In [21]:
print_day(10)

Part 1: 1656
Part 2: 56693912375296


## Day 12

**Task**: Move ship by given rules, calculate distance between starting and ending
positions.

Quite straightforward puzzle: biggest difference between parts is movement of
direction (part 1) / waypoint (part 2), otherwise program logic is the same between
parts. Here, `apply_instructions` function gets direction or waypoint as a keyword
argument `other` and returns the placement of the ship after instructions.

Note that pythons `complex` type offers fine way to represent points and do arithmetics
with them.


In [22]:
def day_12():
    instructions = [(line[0], int(line[1:])) for line in data_to_lines(day=12)]
    directions = {"N": complex(0, 1), "E": complex(1, 0),
                  "S": complex(0, -1), "W": complex(-1, 0)}

    # TODO: fix type errors here
    def turn(degrees: int, direction: complex) -> complex:
        x, y = direction.real, direction.imag
        return direction if degrees % 360 == 0 else turn(degrees - 90, complex(y, -x))

    def move(steps: int, point: complex, direction: complex) -> complex:
        x, y = point.real, point.imag
        dx, dy = direction.real, direction.imag
        return complex(x + steps * dx, y + steps * dy)

    def apply_instructions(other: complex, mode: Mode) -> complex:
        point = complex(0,0)
        for action, value in instructions:
            if action == "R":
                other = turn(degrees=value, direction=other)
            elif action == "L":
                other = turn(degrees=-value, direction=other)
            elif action == "F":
                point = move(steps=value, point=point, direction=other)
            elif mode == "first":
                point = move(steps=value, point=point, direction=directions[action])
            elif mode == "second":
                other = move(steps=value, point=other, direction=directions[action])
        return point

    def solve(mode: Mode) -> int:
        point = (apply_instructions(other=directions["E"], mode=mode) if mode == "first"
                 else apply_instructions(other=complex(10, 1), mode=mode))
        return int(abs(point.real) + abs(point.imag))

    return solve(mode="first"), solve(mode="second")

In [23]:
print_day(12)

Part 1: 1010
Part 2: 52742


## Day 13

**Task**: Given some timestamp, and a bus schedule, what is the earliest time to take
a bus (part 1) / the earliest time when busses depart exactly n minutes after first,
where n is their placement on the list.

For input, ids and diffs between bus id and its placement in schedule are calculated.
This helps on part 2.

The first part requires simple calculation, which gets remainder between the earliest
time and bus ids. The second part, on the other hand, requires some number
theory: [chinese remainder theorem](
https://en.wikipedia.org/wiki/Chinese_remainder_theorem). Idea here is that, there is
one integer $x$ that holds

$\begin{align}
x \equiv a_1 \pmod{n_1} \\
x \equiv a_2 \pmod{n_2} \\
x \equiv a_k \pmod{n_k} \\
\end{align}
$

where $a$ is a list of bus ids and $n$ is a list of differences between id and busses
placement in given schedule. Using the given example, this means that
`1068781 % 0 == 7 % 0`, `1068781 % 1 == 13 % 1`,`1068781 % 4 == 59 % 4` etc.

Sympy library [provides function](
https://docs.sympy.org/latest/modules/ntheory.html?highlight=baby%20step#sympy.ntheory.modular.crt)
to calculate the integer $x$.

In [24]:
def day_13():
    lines = data_to_lines(day=13)
    earliest = int(lines[0])
    bus_ids_and_diffs = [
        (int(bus_id), int(bus_id) - i)
        for i, bus_id in enumerate(lines[1].split(","))
        if bus_id != "x"
    ]

    minutes, bus_id = min(
        (bus_id - earliest % bus_id, bus_id) for bus_id, _ in bus_ids_and_diffs
    )
    first = minutes * bus_id

    bus_ids, diffs = zip(*bus_ids_and_diffs)
    second = crt(bus_ids, diffs)[0]
    
    return first, second

In [25]:
print_day(13)

Part 1: 3269
Part 2: 672754131923874


## Day 25

**Task**: Given two public keys, find out the encryption key they share.

The final day, yeah! Exchange process detailed in the puzzle is a lot like [Diffie-
Hellman key exchange](https://en.wikipedia.org/wiki/Diffie%E2%80%93Hellman_key_exchange)
- in this case, the secret integer (loop size) must be solved.

Transforming process is basically taking $k$th power of the subject number $7$
modulo $20201227$, where $k = $ loop size. Since public key $n$ is given, loop 
size can be found from the equation $7^k \equiv n \pmod{20201227}$. 

Instead of brute forcing loop sizes $1…k$ until correct is found, it's better to
calculate [discrete logarithm](https://en.wikipedia.org/wiki/Discrete_logarithm) of
$n$ to the base $7$ modulo $20201227$. Luckily, Sympy library provides
[suitable function for this](
https://docs.sympy.org/latest/modules/ntheory.html?#sympy.ntheory.residue_ntheory.discrete_log).

In [26]:
def day_25():
    card_key, door_key = data_to_nums(day=25)
    subject_number, modulo, loop_size = 7, 20201227, 0
    
    k = discrete_log(modulo, card_key, subject_number)
    
    return pow(door_key, k, modulo), None


In [27]:
print_day(25)

Part 1: 3286137
Part 2: None


# Timings
I've been trying to optimize solutions to run under 1 seconds each. Here's the report
on 2018 15" Macbook Pro (2,8 Ghz i7):

In [28]:
import time

def timing():
    """Report on timing for all days."""
    print("Day  Secs.")
    print('===  =====')   
    for day in [1,2,3,4,5,6,7,8,9,10,12,13,25]:
        start = time.time()
        run_day(day)
        total = time.time() - start
        print(f"{day:2} {total:6.3f}")

%time timing()

Day  Secs.
===  =====
 1  0.174
 2  0.002
 3  0.001
 4  0.003
 5  0.001
 6  0.005
 7  0.019
 8  0.021
 9  0.009
10  0.001
12  0.002
13  0.001
25  0.001
CPU times: user 234 ms, sys: 9.32 ms, total: 243 ms
Wall time: 243 ms
